<a href="https://colab.research.google.com/github/JD-Lora1/LLM-RAG-youtube-transcript/blob/main/NN_transcript_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb sentence-transformers youtube-transcript-api

Prueba de API

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "8NKUAUt8lzc"

ytt_api = YouTubeTranscriptApi()

transcript = ytt_api.fetch(video_id)
print(transcript)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='good morning', start=46.16, duration=5.28), FetchedTranscriptSnippet(text="so i'll switch now to english uh", start=47.12, duration=7.52), FetchedTranscriptSnippet(text="i think it's the it's the language tedx", start=51.44, duration=4.639), FetchedTranscriptSnippet(text='language first of all', start=54.64, duration=5.2), FetchedTranscriptSnippet(text='andre the rest all the organizers thank', start=56.079, duration=5.761), FetchedTranscriptSnippet(text="you very much for your invitation it's", start=59.84, duration=4.16), FetchedTranscriptSnippet(text='uh with the an enormous pleasure that', start=61.84, duration=3.12), FetchedTranscriptSnippet(text="i'm here", start=64.0, duration=4.32), FetchedTranscriptSnippet(text='uh talking about my dream and', start=64.96, duration=6.08), FetchedTranscriptSnippet(text='trying to spread also the word uh what', start=68.32, duration=5.68), FetchedTranscriptSnippet(text='is my idea what d

In [ ]:
# Extraer los textos y metadatos
texts = []
metadatas = []
for i, entry in enumerate(transcript):
    texts.append(entry.text)  # Usar .text en lugar de ['text']
    metadatas.append({
        'start': entry.start,  # Usar .start en lugar de ['start']
        'duration': entry.duration,  # Usar .duration en lugar of ['duration']
        'order': i  # Para mantener el orden original
    })

# Verificar los resultados
print("Textos extraídos:", texts)
print("Metadatos:", metadatas)

Textos extraídos: ['good morning', "so i'll switch now to english uh", "i think it's the it's the language tedx", 'language first of all', 'andre the rest all the organizers thank', "you very much for your invitation it's", 'uh with the an enormous pleasure that', "i'm here", 'uh talking about my dream and', 'trying to spread also the word uh what', 'is my idea what do i want to do', "the the name of my presentation is it's", 'called blogging for a dream i think you', 'will', "identify what's this this is antarctica", 'this is the most', 'southern most southern continent of the', 'world', 'this will be a pretty light easygoing', 'presentation', 'i hope you enjoy it and', 'this happened two months ago', 'i received an email from from a company', 'a north american company called quark', 'expeditions', 'on this email i had i was', 'they were challenging me something like', 'luis monteiro i know you are', 'interest interested in the antarctica', 'and the', 'things that surround the antarct

ChromaDB

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Inicializar la función de embedding
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Crear el cliente de ChromaDB
chroma_client = chromadb.Client()

# Crear una colección con la función de embedding
collection = chroma_client.create_collection(
    name="youtube_transcripts",
    embedding_function=sentence_transformer_ef
)

In [ ]:
# Generar IDs únicos para cada segmento
ids = [str(i) for i in range(len(texts))]

# Añadir a la colección
collection.add(
    documents=texts,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
# Ejemplo de consulta: buscar segmentos relevantes sobre "elephants"
results = collection.query(
    query_texts=["elephants"],
    n_results=2
)

print("Documentos encontrados:", results['documents'])
print("Metadatos:", results['metadatas'])

Documentos encontrados: [['most amazing animal have you seen a', "animal it's the most"]]
Metadatos: [[{'start': 435.52, 'order': 145, 'duration': 3.28}, {'duration': 7.84, 'order': 143, 'start': 427.68}]]


Modelo Llama

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from youtube_transcript_api import YouTubeTranscriptApi
import torch
from google.colab import userdata


# Configuración inicial
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #"meta-llama/Llama-2-7b-chat-hf"#
token = userdata.get('HFSecret') #Poner Token en collab

tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",
    torch_dtype=torch.float32,
    token=token
)

# Crear una pipeline de generación de texto
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

Device set to use cpu


In [ ]:
# 2. Función para obtener y procesar transcripciones
def get_video_transcription(video_id):
    try:
        transcript = ytt_api.fetch(video_id)
        texts = []
        metadatas = []

        for i, entry in enumerate(transcript):
            texts.append(entry.text)
            metadatas.append({
                'start': entry.start,
                'duration': entry.duration,
                'order': i
            })

        return texts, metadatas
    except Exception as e:
        print(f"Error obteniendo transcripción: {e}")
        return [], []



In [ ]:
# 3. Configurar ChromaDB
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection_name="youtube_transcripts"

try:
    chroma_client.delete_collection(name=collection_name)
    print(f"Colección '{collection_name}' eliminada exitosamente")
except ValueError:
    print(f"La colección '{collection_name}' no existía, se creará nueva")
except Exception as e:
    print(f"Error al eliminar la colección: {e}")

# Crear la colección (nueva o renovada)
collection = chroma_client.create_collection(
    name=collection_name,
    embedding_function=sentence_transformer_ef
)

# 4. Función para agregar transcripción a ChromaDB
def add_transcription_to_chroma(video_id):
    texts, metadatas = get_video_transcription(video_id)

    if texts:
        ids = [f"{video_id}_{i}" for i in range(len(texts))]
        collection.add(
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )
        print(f"Transcripción del video {video_id} agregada a ChromaDB")
        return True
    else:
        print("No se pudo obtener la transcripción")
        return False

Colección 'youtube_transcripts' eliminada exitosamente


In [ ]:
# 5. Función para consultar el RAG y generar respuestas
def ask_about_video(question, video_id, max_length=512):
    # Primero asegurarnos de que la transcripción está en ChromaDB
    if not collection.get(where={"video_id": video_id})['ids']:
        print("Transcripción no encontrada, agregándola...")
        add_transcription_to_chroma(video_id)

    # Buscar contextos relevantes en ChromaDB
    results = collection.query(
        query_texts=[question],
        n_results=2,
        where={"video_id": video_id}  # Filtrar por video específico
    )

    # Combinar los contextos relevantes
    context = " ".join(results['documents'][0])

    # Crear el prompt para TinyLlama
    prompt = f"""
    Basado en la siguiente transcripción de video, responde la pregunta del usuario.

    Transcripción:
    {context}

    Pregunta: {question}

    Respuesta:
    """

    # Generar la respuesta con TinyLlama
    response = llm_pipeline(
        prompt,
        max_length=max_length,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    return response[0]['generated_text'].replace(prompt, "").strip()

In [ ]:
# 6. Función principal para interactuar con el sistema
def video_qa_system():
    print("Sistema de Preguntas y Respuestas sobre Videos")
    print("---------------------------------------------")

    video_id = input("Ingresa el ID del video de YouTube: ")

    # Agregar la transcripción a ChromaDB
    if add_transcription_to_chroma(video_id):
        print("\n¡Transcripción procesada exitosamente!")
        print("Puedes hacer preguntas sobre el video. Escribe 'salir' para terminar.")

        while True:
            question = input("\nTu pregunta: ")
            if question.lower() == 'salir':
                break

            answer = ask_about_video(question, video_id)
            print(f"\nRespuesta: {answer}")

In [ ]:
# Ejemplo de uso con el video "Me at the zoo"
video_id = "8NKUAUt8lzc"#"jNQXAC9IVRw"  # ID de "Me at the zoo"

# Agregar la transcripción a ChromaDB
add_transcription_to_chroma(video_id)

# Hacer algunas preguntas
preguntas = [
    "What are the elephants doing?",
    "What is special about their trunks?",
    "What sound do they make?"
]

for pregunta in preguntas:
    respuesta = ask_about_video(pregunta, video_id)
    print(f"P: {pregunta}")
    print(f"R: {respuesta}\n")

Transcripción del video 8NKUAUt8lzc agregada a ChromaDB
Transcripción no encontrada, agregándola...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Transcripción del video 8NKUAUt8lzc agregada a ChromaDB
P: What are the elephants doing?
R: - Los elefantes están caminando y moviéndose a través de la playa.
    - Los elefantes están paseando y andando por la playa.
    - Los elefantes están caminando y moviéndose en dirección opuesta.
    - Los elefantes están caminando y moviéndose hacia el centro de la playa.
    - Los elefantes están caminando y moviéndose hacia el borde de la playa.
    - Los elefantes están caminando y moviéndose a través de la playa.

Transcripción no encontrada, agregándola...
Transcripción del video 8NKUAUt8lzc agregada a ChromaDB
P: What is special about their trunks?
R: - Especialidad del estómago de sus pezes: su tronco es más largo que el de otras especies y se utiliza para transportar alimentos a los pulmones y los ojos.

Transcripción no encontrada, agregándola...
Transcripción del video 8NKUAUt8lzc agregada a ChromaDB
P: What sound do they make?
R: Sí, ellos suenan.

4. Conversación con una persona so

Modelo desde 0

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para todas las oraciones
sentence_embeddings = embedder.encode(texts)

# Crear etiquetas simuladas (para un proyecto real, necesitarías datos etiquetados)
# Aquí usamos una heurística simple: las oraciones más largas son más importantes
y = np.zeros(len(texts))
for i, text in enumerate(texts):
    if len(text.split()) > 4:  # Oraciones con más de 4 palabras
        y[i] = 1

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    sentence_embeddings, y, test_size=0.2, random_state=42
)

print(f"Dataset: {len(X_train)} entrenamiento, {len(X_test)} test")
print(f"Oraciones importantes: {np.sum(y_train)} en train, {np.sum(y_test)} en test")

Dataset: 238 entrenamiento, 60 test
Oraciones importantes: 165.0 en train, 37.0 en test


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Dimensiones de entrada (tamaño del embedding)
input_dim = sentence_embeddings.shape[1]

# Crear el modelo
model = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Clasificación binaria
])

# Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Mostrar resumen del modelo
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,777 (546.00 KB)

 Trainable params: 139,777 (546.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluar el modelo
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.7221 - loss: 0.6589 - precision: 0.7162 - recall: 1.0000 - val_accuracy: 0.6167 - val_loss: 0.6358 - val_precision: 0.6167 - val_recall: 1.0000
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6868 - loss: 0.5529 - precision: 0.6868 - recall: 1.0000 - val_accuracy: 0.6167 - val_loss: 0.5964 - val_precision: 0.6167 - val_recall: 1.0000
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7164 - loss: 0.4395 - precision: 0.7073 - recall: 1.0000 - val_accuracy: 0.7500 - val_loss: 0.4533 - val_precision: 0.7115 - val_recall: 1.0000
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8965 - loss: 0.2936 - precision: 0.8657 - recall: 1.0000 - val_accuracy: 0.8667 - val_loss: 0.3547 - val_precision: 0.8372 - val_recall: 0.9730
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9742 - loss: 0.1240 - precision: 0.9749 - recall: 0.9868 - val_accuracy: 0.8500 - val_loss: 0.3464 - v

In [ ]:
def generate_extractive_summary(model, texts, embeddings, threshold=0.5):
    """
    Genera un resumen extractivo usando el modelo entrenado
    """
    # Predecir probabilidades
    predictions = model.predict(embeddings)

    # Seleccionar oraciones con probabilidad > threshold
    selected_indices = np.where(predictions > threshold)[0]

    # Ordenar por el orden original
    selected_indices = sorted(selected_indices)

    # Crear el resumen
    summary_sentences = [texts[i] for i in selected_indices]
    summary = " ".join(summary_sentences)

    return summary, selected_indices

# Probar con los datos actuales
summary, selected_indices = generate_extractive_summary(model, texts, sentence_embeddings)
print("Resumen generado:")
print(summary)
print(f"Oraciones seleccionadas: {selected_indices}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Resumen generado:
so i'll switch now to english uh i think it's the it's the language tedx andre the rest all the organizers thank you very much for your invitation it's uh with the an enormous pleasure that uh talking about my dream and trying to spread also the word uh what is my idea what do i want to do the the name of my presentation is it's called blogging for a dream i think you identify what's this this is antarctica southern most southern continent of the this will be a pretty light easygoing i hope you enjoy it and this happened two months ago i received an email from from a company a north american company called quark on this email i had i was they were challenging me something like luis monteiro i know you are interest interested in the antarctica things that surround the antarctic so i challenge you to enter in a where you could win a trip to antarctica this email had something like three four first question to enter in the do you ha

In [ ]:
def summarize_new_video(video_id, model, embedder, threshold=0.5):
    """
    Función completa para resumir un nuevo video
    """
    # 1. Extraer transcripción
    transcript = ytt_api.fetch(video_id)

    # 2. Procesar texto
    new_texts = []
    for entry in transcript:
        new_texts.append(entry.text)

    # 3. Generar embeddings
    new_embeddings = embedder.encode(new_texts)

    # 4. Predecir oraciones importantes
    predictions = model.predict(new_embeddings)

    # 5. Generar resumen
    selected_indices = np.where(predictions > threshold)[0]
    selected_indices = sorted(selected_indices)
    summary_sentences = [new_texts[i] for i in selected_indices]
    summary = " ".join(summary_sentences)

    return summary, new_texts, selected_indices

# Ejemplo de uso
new_video_id = "jNQXAC9IVRw"  # El mismo video de prueba
summary, all_sentences, selected_indices = summarize_new_video(new_video_id, model, embedder)

print("Texto completo:", " ".join(all_sentences))
print("\nResumen extractivo:")
print(summary)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Texto completo: All right, so here we are, in front of the
elephants the cool thing about these guys is that they
have really... really really long trunks and that's cool (baaaaaaaaaaahhh!!) and that's pretty much all there is to
say

Resumen extractivo:
All right, so here we are, in front of the
elephants the cool thing about these guys is that they
have really... and that's cool (baaaaaaaaaaahhh!!) and that's pretty much all there is to
say


Mejorar modelo?

In [ ]:
# Data augmentation para mejorar el entrenamiento
def augment_data(embeddings, labels, num_augmented=1000):
    """
    Aumentar datos sintéticamente
    """
    from sklearn.utils import resample

    # Enfocarse en la clase minoritaria (oraciones importantes)
    important_indices = np.where(labels == 1)[0]
    unimportant_indices = np.where(labels == 0)[0]

    # Sobremuestrear la clase minoritaria
    augmented_important = resample(
        embeddings[important_indices],
        n_samples=num_augmented,
        random_state=42
    )

    augmented_labels = np.ones(num_augmented)

    # Combinar con datos originales
    X_augmented = np.vstack([embeddings, augmented_important])
    y_augmented = np.concatenate([labels, augmented_labels])

    return X_augmented, y_augmented

# Mejorar el modelo con más capas
def create_advanced_model(input_dim):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'f1_score']
    )

    return model